In [19]:
# | default_exp classes.DomoPage


In [20]:
# | exporti
from fastcore.basics import patch_to
from dataclasses import dataclass, field

import asyncio
import httpx

import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoUser as dmdu
import domolibrary.utils.DictDot as util_dd

import domolibrary.routes.page as page_routes


# Page Layout classses

In [21]:
# | export
@dataclass
class PageLayoutTemplate:
    content_key: int
    x: int
    y: int
    width: int
    height: int
    type: str
    virtual: bool
    virtual_appendix: bool
    
    @classmethod
    def _from_json(cls, dd):
        return cls(
            content_key=dd.contentKey,
            x=dd.x,
            y=dd.y,
            width=dd.width,
            height=dd.height,
            type= dd.type,
            virtual=dd.virtual,
            virtual_appendix = dd.virtualAppendix,
           
        )
    
    def get_body(self):
        return {
            "contentKey": self.content_key,
            "x": self.x,
            "y":self.y,
            "width": self.width,
            "height": self.height,
            "type":self.type,
            "virtual":self.virtual,
            "virtualAppendix":self.virtual_appendix
        }

@dataclass
class PageLayoutBackground:
    id: int
    crop_height: int
    crop_width: int
    x: int
    y: str
    data_file_id: int
    image_brightness: int
    image_height: int
    image_width: int
    selected_color: str
    text_color: str
    type: str
    is_darkMode: bool
    alpha: float
    src: str
    
    @classmethod
    def _from_json(cls, dd):
        if dd is not None:
            return cls(
            id=dd.id,
            crop_height=dd.cropHeight,
            crop_width=dd.cropWidth,
            x = dd.x,
            y = dd.y,
            data_file_id = dd.dataFileId,
            image_brightness = dd.imageBrightness,
            image_height = dd.imageHeight,
            image_width = dd.imageWidth,
            selected_color = dd.selectedColor,
            text_color = dd.textColor,
            type = dd.type,
            is_darkMode = dd.darkMode,
            alpha = dd.alpha,
            src = dd.src
            
            )
        else:
            return None

    def get_body(self):
        return {
            "id": self.id,
            "cropHeight": self.crop_height,
            "cropWidth": self.crop_width,
            "x": self.x,
            "y": self.y,
            "dataFileId": self.data_file_id,
            "imageBrightness": self.image_brightness,
            "imageHeight": self.image_height,
            "imageWidth": self.image_width,
            "selectedColor": self.selected_color,
            "textColor": self.text_color,
            "type": self.type,
            "darkMode": self.is_darkMode,
            "alpha": self.alpha,
            "src": self.src
        }

@dataclass
class PageLayoutContent:
    accept_date_filter: bool
    accept_filters: bool
    accept_segments: bool
    card_id: int
    card_urn: str
    compact_interaction_default: bool
    content_key: int
    fit_to_frame: bool
    has_summary: bool
    hide_border: bool
    hide_description: bool
    hide_footer: bool
    hide_margins:bool
    hide_summary: bool
    hide_timeframe: bool
    hide_title: bool
    hide_wrench: bool
    id: int
    summary_number_only:bool
    type: str
    text: str
    background_id : int
    background : PageLayoutBackground
    
    
    
    @classmethod
    def _from_json(cls, dd):
        return cls(
            accept_date_filter=dd.acceptDateFilter,
            accept_filters=dd.acceptFilters,
            accept_segments=dd.acceptSegments,
            card_id=dd.cardId,
            card_urn=dd.cardUrn,
            compact_interaction_default= dd.compactInteractionDefault,
            content_key=dd.contentKey,
            fit_to_frame = dd.fitToFrame,
            has_summary=dd.hasSummary,
            hide_border = dd.hideBorder,
            hide_description= dd.hideDescription,
            hide_footer = dd.hideFooter,
            hide_margins = dd.hideMargins,
            hide_summary = dd.hideSummary,
            hide_timeframe = dd.hideTimeframe,
            hide_title = dd.hideTitle,
            hide_wrench = dd.hideWrench,
            id = dd.id,
            summary_number_only=dd.summaryNumberOnly,
            type = dd.type,
            text = dd.text,
            background_id = dd.backgroundId,
            background = PageLayoutBackground._from_json(
                    dd=dd.background)
            
        )
    
    def get_body(self):
        body = {
            "acceptDateFilter": self.accept_date_filter,
            "acceptFilters": self.accept_filters,
            "acceptSegments":self.accept_segments,
            "cardId": self.card_id,
            "cardUrn": self.card_urn,
            "compactInteractionDefault":self.compact_interaction_default,
            "contentKey":self.content_key,
            "fitToFrame":self.fit_to_frame,
            "hasSummary":self.has_summary,
            "hideBorder":self.hide_border,
            "hideDescription":self.hide_description,
            "hideFooter":self.hide_footer,
            "hideMargins":self.hide_margins,
            "hideSummary":self.hide_summary,
            "hideTimeframe":self.hide_timeframe,
            "hideTitle":self.hide_title,
            "hideWrench":self.hide_wrench,
            "id":self.id,
            "summaryNumberOnly":self.summary_number_only,
            "type":self.type,
            "text":self.text,
            "backgroundId":self.background_id
        }
        
        if self.background is not None:
            body["background"] = self.background.get_body()
        return body
        
    
@dataclass
class PageLayoutStandard:
    aspect_ratio: float
    width: int
    frame_margin: int
    frame_padding: int
    type: str
    template: list[PageLayoutTemplate]
    
    @classmethod
    def _from_json(cls, dd):
        
        obj = cls(
            aspect_ratio=dd.aspectRatio,
            width=dd.width,
            frame_margin=dd.frameMargin,
            frame_padding=dd.framePadding,
            type = dd.type, 
            template = []
        )
            
        if dd.template is not None:
            for template_item in dd.template:
                dc = PageLayoutTemplate._from_json(
                    dd=template_item)
                if dc not in obj.template:
                    obj.template.append(dc)
        return obj
    
@dataclass
class PageLayoutCompact:
    aspect_ratio: float
    width: int
    frame_margin: int
    frame_padding: int
    type: str
    template: list[PageLayoutTemplate]
    
    @classmethod
    def _from_json(cls, dd):
        obj = cls(
            aspect_ratio=dd.aspectRatio,
            width=dd.width,
            frame_margin=dd.frameMargin,
            frame_padding=dd.framePadding,
            type = dd.type,
            template = []
        )
        if dd.template is not None:
            for template_item in dd.template:
                dc = PageLayoutTemplate._from_json(
                    dd=template_item)
                if dc not in obj.template:
                    obj.template.append(dc)
        return obj
    
@dataclass
class PageLayout:
    id: str
    page_id: str
    is_print_friendly: bool
    is_enabled: bool
    is_dynamic: bool
    has_page_breaks : bool
    content : list[PageLayoutContent]
    standard : PageLayoutStandard
    compact : PageLayoutCompact
    background: PageLayoutBackground
    
    @classmethod
    def _from_json(cls, dd):
    
        obj= cls(
            id=dd.layoutId,
            page_id=dd.pageUrn,
            is_print_friendly=dd.printFriendly,
            is_enabled=dd.enabled,
            is_dynamic=dd.isDynamic,
            content = [],
            has_page_breaks = dd.hasPageBreaks,
            standard = PageLayoutStandard._from_json(
                    dd=dd.standard),
            compact = PageLayoutCompact._from_json(
                    dd=dd.compact),
            background =PageLayoutBackground._from_json(
                    dd=dd.background)
        )
        if dd.content is not None:
            for content_item in dd.content:
                dc = PageLayoutContent._from_json(
                    dd=content_item)
                if dc not in obj.content:
                    obj.content.append(dc)
        return obj
        
    
    @classmethod
    def generate_new_background_body (cls):
        background_body= {
         
            "selectedColor": "#EEE000",
            "textColor": "#4A4A4A",
            "type": "COLOR",
            "darkMode": False,
            "alpha": 1
        }
        
        return background_body

    
    def get_body (self):
        body = {
        "layoutId": self.id,
        "pageUrn": self.page_id,
        "printFriendly" : self.is_print_friendly,
        "enabled": self.is_enabled,
        "isDynamic": self.is_dynamic,
        "hasPageBreaks": self.has_page_breaks,
        "standard" : {
            "aspectRatio":self.standard.aspect_ratio,
            "width": self.standard.width,
            "frameMargin": self.standard.frame_margin,
            "framePadding": self.standard.frame_padding,
            "type": self.standard.type
            },
        "compact" : {
            "aspectRatio":self.compact.aspect_ratio,
            "width": self.compact.width,
            "frameMargin": self.compact.frame_margin,
            "framePadding": self.compact.frame_padding,
            "type": self.compact.type
            }
         }
        if self.background is not None:
            body["background"] = self.background.get_body()

        if  self.content ==[] or self.content is None:
            body["content"] = []
        else:
            temp_list = []
            for content_item in self.content:
                temp_list.append(content_item.get_body())
            body["content"]= temp_list
            
            
        if self.standard.template is None or self.standard.template==[]:
            body["standard"]["template"]=[]
        else:
            temp_list = []
            for template_item in self.standard.template:
                temp_list.append(template_item.get_body())
            body["standard"]["template"]= temp_list
            
        if self.compact.template is None or self.compact.template==[]:
            body["compact"]["template"]=[]
        else:
            temp_list = []
            for template_item in self.compact.template:
                temp_list.append(template_item.get_body())
            body["compact"]["template"]= temp_list
        return body


In [22]:
# | export
@dataclass
class DomoPage:
    id: str
    title: str = None
    parent_page_id: str = None
    top_page_id: str = None
    auth: dmda.DomoAuth = field(default = None , repr = False)
    owners: list = field(default_factory=list)
    cards: list = field(default_factory=list)
    collections: list = field(default_factory=list)
    children: list = field(default_factory=list)
    is_locked : bool = None
    layout: PageLayout = None

    def display_url(self):
        return f"https://{self.auth.domo_instance}.domo.com/page/{self.id}"

In [23]:
# | exporti
@patch_to(DomoPage)
async def _get_domo_users(self, user_ls: [dict]):
    import domolibrary.classes.DomoUser as dmu
    import domolibrary.classes.DomoGroup as dmg
    tasks = []

    for user in user_ls:
        if user.get('type') =='USER':
            tasks.append(dmu.DomoUser.get_by_id(user_id=user.get('id') or user.get('userId'), auth=self.auth))
        if user.get('type') == 'GROUP':
            tasks.append(dmg.DomoGroup.get_by_id(group_id=user.get('id') or user.get('groupId'), auth=self.auth))

    return await asyncio.gather( *tasks)


@patch_to(DomoPage, cls_method=True)
async def _from_bootstrap(cls: DomoPage, page_obj, auth: dmda.DomoAuth = None):

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    domo_page = cls(id=dd.id, title=dd.title, auth=auth)

    if isinstance(dd.owners, list) and len(dd.owners) > 0:
        domo_page.owners = await domo_page._get_domo_users(page_obj.get('owners'))

    if isinstance(dd.children, list) and len(dd.children) > 0:
        domo_page.children = [
            cls._from_bootstrap(page_obj=child_dd, auth=auth)
            for child_dd in dd.children
            if child_dd.type == "page"
        ]

        [print(other_dd) for other_dd in dd.children
            if other_dd.type != "page"]

    return domo_page


#### sample _from_bootstrap_routes

In [24]:
import os
import domolibrary.routes.bootstrap as bootstrap_routes

auth = dmda.DomoFullAuth(
    domo_instance='domo-community',
    domo_password = os.environ['DOJO_PASSWORD'],
    domo_username= os.environ['DOMO_USERNAME'])

res = await bootstrap_routes.get_bootstrap_pages(auth = auth)

page_obj = res.response[10]
await DomoPage._from_bootstrap(page_obj, auth = auth)

DomoPage(id='1642445256', title='Marketing Calendar Demo', parent_page_id=None, top_page_id=None, owners=[DomoUser(id='204012057', title='Business Insights Analyst', department='', display_name='DuncanDomo', email_address='duncan.wood22@gmail.com', role_id=2, phone_number='', web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], collections=[], children=[], is_locked=None, layout=None)

In [25]:
# | exporti

@patch_to(DomoPage, cls_method=True)
async def _from_content_stacks_v3(cls: DomoPage, page_obj, auth: dmda.DomoAuth = None):
    # import domolibrary.classes.DomoCard as dc

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    pg = cls(
        id=dd.id,
        title=dd.title,
        parent_page_id=dd.page.parentPageId,
        collections=dd.collections,
        auth=auth
    )
    if hasattr(dd, 'pageLayoutV4') and dd.pageLayoutV4 is not None:
                dd_layout = dd.pageLayoutV4
                pg.layout =PageLayout._from_json(
                    dd=dd.pageLayoutV4)
    
    if dd.page.owners and len(dd.page.owners) > 0:
        pg.owners = await pg._get_domo_users(page_obj.get('page').get('owners'))

    # if dd.cards and len(dd.cards) > 0:
    #     pg.cards = await asyncio.gather(
    #         *[dc.DomoCard.get_from_id(id=card.id, auth=auth) for card in dd.cards])

    return pg


@patch_to(DomoPage, cls_method=True)
async def get_by_id(cls: DomoPage,
                    page_id: str,
                    auth: dmda.DomoAuth,
                    return_raw: bool = False, 
                    debug_api: bool = False,
                    include_layout: bool = False):

    res = await page_routes.get_page_by_id(auth=auth, page_id=page_id, debug_api=debug_api, include_layout=include_layout)

    if return_raw:
        return res
        
    if not res.status == 200:
        return

    pg = await cls._from_content_stacks_v3(page_obj=res.response, auth=auth)

    return pg


#### sample implementation of get_from_id

In [26]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
domo_page = await DomoPage.get_by_id(page_id=page_id, auth=token_auth, return_raw=False, include_layout=True)

domo_page.layout


PageLayout(id=1668051644, page_id='1761849366', is_print_friendly=True, is_enabled=True, is_dynamic=False, has_page_breaks=False, content=[PageLayoutContent(accept_date_filter=True, accept_filters=True, accept_segments=True, card_id=None, card_urn=None, compact_interaction_default=True, content_key=0, fit_to_frame=False, has_summary=False, hide_border=False, hide_description=True, hide_footer=False, hide_margins=False, hide_summary=False, hide_timeframe=False, hide_title=False, hide_wrench=False, id=21295, summary_number_only=False, type='HEADER', text='How did education impact povert', background_id=None, background=None), PageLayoutContent(accept_date_filter=True, accept_filters=True, accept_segments=True, card_id=None, card_urn=None, compact_interaction_default=True, content_key=1, fit_to_frame=False, has_summary=False, hide_border=False, hide_description=True, hide_footer=False, hide_margins=False, hide_summary=False, hide_timeframe=False, hide_title=False, hide_wrench=False, id=21

In [27]:
#| exporti

@patch_to(DomoPage,cls_method=True)
async def _from_adminsummary(cls, page_obj, auth :dmda.DomoAuth):

    import domolibrary.classes.DomoCard as dmc

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    pg = cls(
        id=dd.id or dd.pageId,
        title=dd.title or dd.pageTitle,
        parent_page_id=dd.parentPageId,
        top_page_id = dd.topPageId,
        collections=dd.collections,
        is_locked = dd.locked,
        auth=auth,
    )

    if dd.page and dd.page.owners and len(dd.page.owners) > 0:
        pg.owners = await pg._get_domo_users(page_obj.get('page').get('owners'))

    elif dd.owners and len(dd.owners) > 0:
        pg.owners = await pg._get_domo_users(page_obj.get('owners'))


    if dd.cards and len(dd.cards) > 0:
        pg.cards = await asyncio.gather(
            *[dmc.DomoCard.get_from_id(id=card.id, auth=auth) for card in dd.cards])

    return pg


In [28]:
# | export
@patch_to(DomoPage)
async def get_accesslist(self,
                         auth: dmda.DomoAuth = None,
                         is_expand_users: bool = False,
                         return_raw: bool = False,
                         debug_api: bool = False):

    auth = auth or self.auth

    res = await page_routes.get_page_access_list(auth=auth,
                                                is_expand_users=is_expand_users,
                                                page_id=self.id,
                                                debug_api=debug_api
                                                )

    if return_raw:
        return res

    if not res.is_success :
        raise Exception('error getting access list')

    import domolibrary.classes.DomoUser as dmu
    import domolibrary.classes.DomoGroup as dmg
    
    tasks = await asyncio.gather( 
        dmu.DomoUsers.by_id(user_ids=[user.get('id') for user in res.response.get('users')], only_allow_one=False, auth=auth),
        * [ dmg.DomoGroup.get_by_id( group_id = group.get('id'), auth = auth) for group in res.response.get('groups')]
    )
    
    res.response.update({'users': tasks[0], 'groups': tasks[1:]})

    return res.response


#### sample get_accesslist

In [29]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
domo_page = DomoPage(id=page_id, auth=token_auth)

await domo_page.get_accesslist(is_expand_users=True, return_raw=False, debug_api=False)


{'users': [DomoUser(id='1216550715', title=None, department=None, display_name='8:26 - go to sleep', email_address='test4@domo.com', role_id=2, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
  DomoUser(id='1628021317', title=None, department=None, display_name='Aaron Dean', email_address='aaron.dean@rxa.io', role_id=2097317660, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
  DomoUser(id='1542225148', title=None, department=None, display_name='Adam Landefeld', email_address='Adam.Landefeld@domo.com', role_id=2097317660, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to

In [30]:
#| export

@dataclass
class DomoPages:
    pass

In [31]:
# | exporti

@patch_to(DomoPages, cls_method=True)
async def get_pages(cls: DomoPages, auth=dmda.DomoAuth,
                    return_raw: bool = False,
                    debug_loop: bool = False, debug_api: bool = False, session: httpx.AsyncClient = None):
    is_close_session = False if session else True

    session = session or httpx.AsyncClient()

    try:
        res = await page_routes.get_pages_adminsummary(auth=auth, debug_loop=False, debug_api=False, session=session)

        if return_raw: 
            return res
            
        if not res.is_success:
            raise Exception('unable to retrieve pages')
        

        return await asyncio.gather(*[DomoPage._from_adminsummary(page_obj, auth=auth) for page_obj in res.response])

    finally:
        if is_close_session:
            await session.aclose()


In [32]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoPages.get_pages(auth=token_auth, return_raw=False)


[DomoPage(id=1316566624, title='20210623_TRAINING_DomoStats Activity Log App', parent_page_id=127044793, top_page_id=522373865, owners=[DomoUser(id='1893952720', title=None, department='Business Improvement', display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=1, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], collections=None, children=[], is_locked=False, layout=None),
 DomoPage(id=384424178, title='75th Percentile Test', parent_page_id=None, top_page_id=None, owners=[DomoUser(id='1898323170', title=None, department=None, display_name='Creed Smith', email_address='creed.smith@domo.com', role_id=2, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, 

In [33]:


#     @classmethod
#     async def get_cards(cls, auth, page_id, debug: bool = False, session: httpx.AsyncClient = None):
#         try:
#             import Library.DomoClasses.DomoCard as dc
#             close_session = False if session else True

#             if not session:
#                 session = httpx.AsyncClient()

#             res = await page_routes.get_page_definition(auth=auth, page_id=page_id, debug=debug, session=session)

#             if res.status == 200:
#                 json = res.response

#                 card_list = [dc.DomoCard(id=card.get(
#                     'id'), auth=auth) for card in json.get('cards')]

#                 return card_list

#             else:
#                 return None

#         finally:
#             if close_session:
#                 await session.aclose()

#     async def get_datasets(auth, page_id, debug: bool = False, session: httpx.AsyncClient = None):
#         try:
#             import Library.DomoClasses.DomoDataset as dmds
#             close_session = False if session else True

#             if not session:
#                 session = httpx.AsyncClient()

#             res = await page_routes.get_page_definition(auth=auth, page_id=page_id, debug=debug, session=session)

#             if res.status == 200:
#                 json = res.response

#                 dataset_ls = [card.get('datasources')
#                               for card in json.get('cards')]

#                 return [dmds.DomoDataset(id=ds.get('dataSourceId'), auth=auth) for ds_ls in dataset_ls for ds in ds_ls]

#             else:
#                 return None

#         finally:
#             if close_session:
#                 await session.aclose()

In [34]:
# | exporti
from datetime import datetime
from utils import convert

@patch_to(DomoPage, cls_method=True)
async def update_layout(cls,
                         auth: dmda.DomoAuth,
                         body: dict,
                         layout_id: str,
                         debug_api: bool = False):
    datetime_now = datetime.now()
    start_time_epoch = convert.convert_datetime_to_epoch_millisecond(
            datetime_now)
        
    res_writelock = await page_routes.put_writelock(auth=auth, 
                                                        layout_id = layout_id,
                                                        user_id=auth.user_id,
                                                         epoch_time = start_time_epoch)
    if res_writelock.status == 200:
        
        res = await page_routes.update_page_layout(auth=auth,
                                          body=body,
                                       layout_id = layout_id,
                                       debug_api=debug_api)

    
        if res.status != 200:
            return False
            
        res_writelock = await page_routes.delete_writelock(auth=auth, 
                                                        layout_id = layout_id)
        if res_writelock.status != 200:
            return False
            
    else:
        return False

    return True

# Sample update page layout by background color

In [35]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
domo_page = await DomoPage.get_by_id(page_id=page_id, auth=token_auth, return_raw=False, include_layout=True)


body = domo_page.layout.get_body()

if not hasattr(body, 'background'):
    new_background_body =  PageLayout.generate_new_background_body()
    body["background"] = new_background_body
            
body["background"]["selectedColor"] ="#FF0000"
auth = dmda.DomoFullAuth(
    domo_instance='domo-community',
    domo_password = os.environ['DOJO_PASSWORD'],
    domo_username= os.environ['DOMO_USERNAME'])

res = await DomoPage.update_layout(auth=auth, 
                                          body = body,
                                          layout_id = domo_page.layout.id)
print (res)

True


In [36]:
# | hide
import nbdev

nbdev.nbdev_export()

Bad pipe message: %s [b'\x1d^\x94\xcb\x93\xcd\xe4\xee\xe4\xcbZ\xf9\x8294\xf65\n\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0', b"a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0"]
Bad pipe message: %s [b'\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00']
Bad pipe message: %s [b'#\xf0\xd18<\xd9\xb1SLeI\xe5\x13\\;\xc6+]\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00', b"\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0\x13\x003\x002\x00\x9a\x00\x99\x00E\x00D\xc0\x07\x